# COUNT NULL VALUES FOR VIDEOS THAT COULN'T BE ANALYZED

In [1]:
import pickle as pkl
import numpy as np

with open('video_features/vector_features/train_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

vectores_zero = []

for i in range(len(features)):
    if np.all(features[i]["feature_vector"] == 0):
        vectores_zero.append(i)

print(len(vectores_zero))

60


# GET THE LABELS FOR THE DATA

In [2]:
from sklearn.preprocessing import LabelEncoder

with open('video_features/vector_features/train_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

labels = set()

for i in range(len(features)):
    labels.add(features[i]["label"])

le = LabelEncoder()
le.fit(list(labels))

LabelEncoder()

# REDUCE DIMENSIONALITY OF THE DATA WITH PCA

In [3]:
from sklearn.decomposition import PCA

# TRAIN DATA WITH PCA

with open('video_features/vector_features/train_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

X_train = []
y_train = []

for i in range(len(features)):
    if not np.all(features[i]["feature_vector"] == 0):
        X_train.append(features[i]["feature_vector"])
        y_train.append(features[i]["label"])

X_train = np.array(X_train)
y_train = le.transform(y_train)

pca = PCA(n_components=0.9)
X_train_reducido = pca.fit_transform(X_train)

print(X_train_reducido.shape)

# VALIDATION DATA WITH PCA

with open('video_features/vector_features/val_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

X_val = []
y_val = []

for i in range(len(features)):
    if not np.all(features[i]["feature_vector"] == 0):
        X_val.append(features[i]["feature_vector"])
        y_val.append(features[i]["label"])

X_val = np.array(X_val)
y_val = le.transform(y_val)

X_val_reducido = pca.transform(X_val)

print(X_val_reducido.shape)

(5372, 229)
(421, 229)


# REDUCE DIMENTIONALITY OF THE DATA WITH LDA

In [18]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# TRAIN DATA WITH LDA

with open('video_features/vector_features/train_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

X_train = []
y_train = []

for i in range(len(features)):
    if not np.all(features[i]["feature_vector"] == 0):
        X_train.append(features[i]["feature_vector"])
        y_train.append(features[i]["label"])

X_train = np.array(X_train)
y_train = le.transform(y_train)


lda = LDA(n_components=9)
X_train_reducido = lda.fit_transform(X_train, y_train)

# VALIDATION DATA WITH LDA

with open('video_features/vector_features/val_10_fixed_features.pkl', 'rb') as f:
    val_features = pkl.load(f)

# Preparar los datos de validación
X_val = []
y_val = []
for i in range(len(val_features)):
    if not np.all(val_features[i]["feature_vector"] == 0):
        X_val.append(val_features[i]["feature_vector"])
        y_val.append(val_features[i]["label"])

X_val = np.array(X_val)
y_val = le.transform(y_val)

# Transformar los datos de validación con el mismo LDA ajustado
X_val_reducido = lda.transform(X_val)

# REDUCE DIMENTIONALITY OF THE DATA WITH RANDOM PROJECTION

In [33]:
from sklearn.random_projection import GaussianRandomProjection

n = features[0]["feature_vector"].shape[0]
print(n)
epsilon = 0.5
k = int(np.ceil(20*np.log2(n) / (epsilon**2)))
print(k)

# TRAIN DATA WITH RANDOM PROJECTION

with open('video_features/vector_features/train_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

X_train = []
y_train = []

for i in range(len(features)):
    if not np.all(features[i]["feature_vector"] == 0):
        X_train.append(features[i]["feature_vector"])
        y_train.append(features[i]["label"])

X_train = np.array(X_train)
y_train = le.transform(y_train)

grp = GaussianRandomProjection(n_components=k)
X_train_reducido = grp.fit_transform(X_train)

# VALIDATION DATA WITH RANDOM PROJECTION

with open('video_features/vector_features/val_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

X_val = []
y_val = []

for i in range(len(features)):
    if not np.all(features[i]["feature_vector"] == 0):
        X_val.append(features[i]["feature_vector"])
        y_val.append(features[i]["label"])

X_val = np.array(X_val)
y_val = le.transform(y_val)

X_val_reducido = grp.transform(X_val)


1024
800


# REDUCE DIMENTIONALITY OF THE DATA WITH SVD

In [112]:
from sklearn.decomposition import TruncatedSVD

# TRAIN DATA WITH SVD

with open('video_features/vector_features/train_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

X_train = []
y_train = []

for i in range(len(features)):
    if not np.all(features[i]["feature_vector"] == 0):
        X_train.append(features[i]["feature_vector"])
        y_train.append(features[i]["label"])

X_train = np.array(X_train)
y_train = le.transform(y_train)

svd = TruncatedSVD(n_components=9)
X_train_reducido = svd.fit_transform(X_train)

# VALIDATION DATA WITH SVD

with open('video_features/vector_features/val_10_fixed_features.pkl', 'rb') as f:
    features = pkl.load(f)

X_val = []
y_val = []

for i in range(len(features)):
    if not np.all(features[i]["feature_vector"] == 0):
        X_val.append(features[i]["feature_vector"])
        y_val.append(features[i]["label"])

X_val = np.array(X_val)

X_val_reducido = svd.transform(X_val)


# USE KMEANS TO CLUSTER THE DATA

In [109]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score

kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(X_train_reducido)

# Predecir los clusters en los datos de validación
y_val_pred = kmeans.predict(X_val_reducido)

# Calcular e imprimir el Silhouette Score y el Adjusted Rand Index
sil_score = silhouette_score(X_val_reducido, y_val_pred)
ari_score = adjusted_rand_score(y_val, y_val_pred)

print(f"Dimensiones reducidas: {X_train_reducido.shape}")
print(f"Silhouette Score: {sil_score}")
print(f"Adjusted Rand Index: {ari_score}")

Dimensiones reducidas: (5372, 9)
Silhouette Score: 0.24765607058295788
Adjusted Rand Index: 0.2775394298721413


# USE AGGLOMERATIVE CLUSTERING TO CLUSTER THE DATA

In [110]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score

agg = AgglomerativeClustering(n_clusters=10)
agg.fit(X_train_reducido)

# Predecir los clusters en los datos de validación
y_val_pred = agg.fit_predict(X_val_reducido)

# Calcular e imprimir el Silhouette Score y el Adjusted Rand Index
sil_score = silhouette_score(X_val_reducido, y_val_pred)
ari_score = adjusted_rand_score(y_val, y_val_pred)

print(f"Dimensiones reducidas: {X_train_reducido.shape}")
print(f"Silhouette Score: {sil_score}")
print(f"Adjusted Rand Index: {ari_score}")


Dimensiones reducidas: (5372, 9)
Silhouette Score: 0.34485849456802836
Adjusted Rand Index: 0.15208075273450858


# USE DENSIY BASED CLUSTERING TO CLUSTER THE DATA

In [113]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, adjusted_rand_score

dbscan = DBSCAN(eps=5, min_samples=10)
dbscan.fit(X_train_reducido)

# Predecir los clusters en los datos de validación
y_val_pred = dbscan.fit_predict(X_val_reducido)

# Calcular e imprimir el Silhouette Score y el Adjusted Rand Index
sil_score = silhouette_score(X_val_reducido, y_val_pred)
ari_score = adjusted_rand_score(y_val, y_val_pred)

print(f"Dimensiones reducidas: {X_train_reducido.shape}")
print(f"Silhouette Score: {sil_score}")
print(f"Adjusted Rand Index: {ari_score}")

Dimensiones reducidas: (5372, 9)
Silhouette Score: 0.8204318550791023
Adjusted Rand Index: 5.5223194292039695e-05
